In [1]:
import numpy as np
from botasaurus.request import request, Request
from botasaurus_requests.response import Response
import json
from typing import Union, Sequence
from pymongo import MongoClient
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
API_PREFIX = 'https://api.sofascore.com/api/v1'
league_id = "8" # Identificador de Laliga
season_id = "61643"


LEAGUE_STATS_FIELDS = [
    'goals', 'yellowCards', 'redCards', 'groundDuelsWon', 'groundDuelsWonPercentage',
    'aerialDuelsWon', 'aerialDuelsWonPercentage', 'successfulDribbles',
    'successfulDribblesPercentage', 'tackles', 'assists', 'accuratePassesPercentage',
    'totalDuelsWon', 'totalDuelsWonPercentage', 'minutesPlayed', 'wasFouled', 'fouls',
    'dispossessed', 'possesionLost', 'appearances', 'started', 'saves', 'cleanSheets',
    'savedShotsFromInsideTheBox', 'savedShotsFromOutsideTheBox',
    'goalsConcededInsideTheBox', 'goalsConcededOutsideTheBox', 'highClaims',
    'successfulRunsOut', 'punches', 'runsOut', 'accurateFinalThirdPasses',
    'bigChancesCreated', 'accuratePasses', 'keyPasses', 'accurateCrosses',
    'accurateCrossesPercentage', 'accurateLongBalls', 'accurateLongBallsPercentage',
    'interceptions', 'clearances', 'dribbledPast', 'bigChancesMissed', 'totalShots',
    'shotsOnTarget', 'blockedShots', 'goalConversionPercentage', 'hitWoodwork', 'offsides',
    'expectedGoals', 'errorLeadToGoal', 'errorLeadToShot', 'passToAssist'
]
concatenated_fields = '%2C'.join(LEAGUE_STATS_FIELDS)

comps = {
    # European continental club comps
    'Champions League': 7, 'Europa League': 679, 'Europa Conference League': 17015,
    # European domestic leagues
    'EPL': 17, 'La Liga': 8, 'Bundesliga': 35, 'Serie A': 23, 'Ligue 1': 34,
    # South America
    'Argentina Liga Profesional': 155, 'Argentina Copa de la Liga Profesional': 13475,
    'Liga 1 Peru': 406, "Copa Libertadores": 384,
    # USA
    'MLS': 242, 'USL Championship': 13363, 'USL1': 13362, 'USL2': 13546,
    # Middle East
    "Saudi Pro League": 955,
    # Men's international comps
    'World Cup': 16, 'Euros': 1, 'Gold Cup': 140,
    # Women's international comps
    "Women's World Cup": 290
}

In [3]:
@request(output=None, create_error_logs=False)
def botasaurus_get(req: Request, url: str) -> Response:
    """ General purpose "get" function that uses Botasaurus.
    
    Parameters
    ----------
    req : botasaurus.request.Request
        The request object provided by the botasaurus decorator
    url : str
        The URL to request
        
    Returns
    -------
    botasaurus_requests.response.Response
        The response from the request
    """
    if not isinstance(url, str):
        raise TypeError('`url` must be a string.')
    resp = req.get(url)
    return resp

In [10]:
def get_valid_seasons(league: str) -> dict:
    """ Returns the valid seasons and their IDs for the given league

    Parameters
    ----------
    league : str
        League to get valid seasons for. See comps ScraperFC.Sofascore for valid leagues.
    
    Returns
    -------
    seasons : dict
        Available seasons for the league. {season string: season ID, ...}
    """
        
    response = botasaurus_get(f'{API_PREFIX}/unique-tournament/{comps[league]}/seasons/')
    seasons = dict([(x['year'], x['id']) for x in response.json()['seasons']])
    return seasons

In [11]:
def get_match_dicts(year: str, league:str) -> Sequence[dict]:
    """ Returns the matches from the Sofascore API for a given league season.

    Parameters
    ----------
    year : str
        See the :ref:`sofascore_year` `year` parameter docs for details.
    league : str
        League to get valid seasons for. See comps ScraperFC.Sofascore for valid leagues.
    
    Returns
    -------
    matches : list of dict
        Each element being a single game of the competition
    """
    valid_seasons = get_valid_seasons(league)

    matches = list()
    i = 0
    while 1:
        response = botasaurus_get(
            f'{API_PREFIX}/unique-tournament/{comps[league]}/season/{valid_seasons[year]}/' +
            f'events/last/{i}'
        )
        if response.status_code != 200:
            break
        matches += response.json()['events']
        i += 1

    return matches

In [12]:
partidos_en_temporada = get_match_dicts("24/25", "La Liga")

print(json.dumps(partidos_en_temporada, indent=4))

KeyError: 'seasons'